In [1]:
# Install necessary packages (only run if not already installed)
# !pip install nltk

In [3]:
# NLP Imports
import nltk
#nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [4]:
# Other Imports
import numpy as np
import random
import json
import pickle
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical

In [5]:
# Load and preprocess intents
with open('intents.json') as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            bag.append(1 if w in wrds else 0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [21]:
# Define the model using Keras
try:
    model = load_model("chatbot_model.keras")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accurac #compile to suppress warningsy'])
except:
    model = Sequential()
    model.add(Input(shape=(len(training[0]),)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(len(output[0]), activation="softmax"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(training, output, epochs=200, batch_size=8)
    model.save("chatbot_model.keras")

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.1391 - loss: 2.2007
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2187 - loss: 2.2170
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1853 - loss: 2.1928
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1978 - loss: 2.1668
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1520 - loss: 2.1800
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1728 - loss: 2.2078
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2437 - loss: 2.1348
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2062 - loss: 2.1701
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2191 - loss: 2.1491
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2524 - loss: 2.1267
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2274 - loss: 2.1322
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1233 - lo

In [23]:
# Function to convert user input into bag-of-words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

In [25]:
# Chat function
def chat():
    print("Start talking with the bot! Type 'quit' to stop.")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))[0]
        results_index = np.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]
            print("Bot:", random.choice(responses))
        else:
            print("Bot: I didn't quite understand that. Try again.")

In [ ]:
# To start chatting, run:
chat()

Start talking with the bot! Type 'quit' to stop.


You:  What are your hours of operation?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Bot: We're open every day 9am-9pm


You:  Where can we locate you?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Bot: We are on the intersection of London Alley and Bridge Avenue.


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Bot: Yes, we provide home delivery through UBER Eats and Zomato?


You:  Do you accept Credit Card?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Bot: We accept VISA, Mastercard and AMEX


You:  What are all the different main course options you have?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Bot: You can check various delicacies given in the food menu at www.mymenu.com
